In [1]:
import pandas as pd

# Load the CSV
df = pd.read_csv("../../data/aqi.csv")

# Convert and extract year
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['year'] = df['date'].dt.year

# Filter relevant years, cities, and parameters
target_years = [2022, 2023, 2024, 2025]
target_cities = [
    "Delhi", "Mumbai", "Chennai", "Kolkata", "Bengaluru", "Hyderabad", "Ahmedabad",
    "Pune", "Noida", "Bhopal", "Gurugram", "Jaipur", "Indore", "Nagpur",
    "Brynihat", "Begusarai", "Ghaziabad"
]
df = df[df['year'].isin(target_years) & df['area'].isin(target_cities)]
df = df[df['prominent_pollutants'].isin(['PM2.5', 'PM10'])]

# Group and calculate average per city/year/parameter
avg_df = df.groupby(['area', 'year', 'prominent_pollutants'])['aqi_value'].mean().reset_index()

# Create separate columns for each year/parameter
pivot_df = avg_df.pivot_table(index='area', columns=['prominent_pollutants', 'year'], values='aqi_value')

# Flatten multi-level column index
pivot_df.columns = [f"{param.upper()}_{year}" for param, year in pivot_df.columns]

# Reset index to make 'city' a column
pivot_df.reset_index(inplace=True)
pivot_df.rename(columns={'area': 'Area'}, inplace=True)

# Sort columns logically
ordered_cols = ['Area'] + sorted([col for col in pivot_df.columns if col != 'Area'])
pivot_df = pivot_df[ordered_cols]

# Display
print(pivot_df.round(2))

# Optional: Save to CSV
pivot_df.to_csv("ncap_wide_format_summary.csv", index=False)


         Area  PM10_2022  PM10_2023  PM10_2024  PM10_2025  PM2.5_2022  \
0   Ahmedabad      95.00      93.66      67.96      93.00      178.88   
1   Begusarai      94.52     206.09     166.75     126.30      283.54   
2   Bengaluru      62.52      66.31      72.21      79.58      128.44   
3      Bhopal     117.13      90.23      91.15     115.76      216.55   
4     Chennai      70.40      70.00      67.86      71.28      159.67   
5       Delhi     252.67     156.16     166.98     236.61      313.41   
6   Ghaziabad     204.40     154.23     127.19      93.25      287.59   
7    Gurugram     136.68      97.40     104.67     134.67      267.48   
8   Hyderabad      85.43      80.80      64.79      85.34      116.91   
9      Indore     108.94      97.97      84.96      85.23      159.00   
10     Jaipur     116.13     103.76     118.59     129.16      189.75   
11    Kolkata      70.25      63.91      80.39      79.41      235.69   
12     Mumbai     107.50      88.02      60.46     

In [2]:
import pandas as pd

# Load the wide-format CSV
df = pd.read_csv("ncap_wide_format_summary.csv")

# Display current columns to debug (optional)
print("Columns in your CSV:", df.columns.tolist())

# Fix: Match actual column names
# Look for columns like 'PM2.5_2022' or 'PM25_2022'
pm25_2022 = next((col for col in df.columns if 'PM2.5' in col.upper() and '2022' in col), None)
pm25_2025 = next((col for col in df.columns if 'PM2.5' in col.upper() and '2025' in col), None)

pm10_2022 = next((col for col in df.columns if 'PM10' in col.upper() and '2022' in col), None)
pm10_2025 = next((col for col in df.columns if 'PM10' in col.upper() and '2025' in col), None)

# Safety check
print(f"Using columns: {pm25_2022}, {pm25_2025} for PM2.5")
print(f"Using columns: {pm10_2022}, {pm10_2025} for PM10")

# Create reduction columns safely
df['PM25_%_Reduction'] = ((df[pm25_2022] - df[pm25_2025]) / df[pm25_2022]) * 100
df['PM10_%_Reduction'] = ((df[pm10_2022] - df[pm10_2025]) / df[pm10_2022]) * 100

df = df.round(2)

# Conditional coloring function
def highlight_reduction(val):
    if pd.isna(val):
        return ''
    elif val >= 20:
        return 'background-color: lightgreen'  # ≥20% = ✅
    elif val >= 10:
        return 'background-color: khaki'       # 10–20% = ⚠️
    else:
        return 'background-color: lightcoral'  # <10% = ❌

# Apply to reduction columns
styled = df.style.map(highlight_reduction, subset=['PM25_%_Reduction', 'PM10_%_Reduction'])

# Save to Excel
styled.to_excel("ncap_summary_colored.xlsx", index=False, engine='openpyxl')

# Show styled DataFrame (in Jupyter or notebook)
styled


Columns in your CSV: ['Area', 'PM10_2022', 'PM10_2023', 'PM10_2024', 'PM10_2025', 'PM2.5_2022', 'PM2.5_2023', 'PM2.5_2024', 'PM2.5_2025']
Using columns: PM2.5_2022, PM2.5_2025 for PM2.5
Using columns: PM10_2022, PM10_2025 for PM10


,Area,PM10_2022,PM10_2023,PM10_2024,PM10_2025,PM2.5_2022,PM2.5_2023,PM2.5_2024,PM2.5_2025,PM25_%_Reduction,PM10_%_Reduction
0,Ahmedabad,95.000000,93.660000,67.960000,93.000000,178.880000,144.840000,142.480000,145.610000,18.600000,2.110000
1,Begusarai,94.520000,206.090000,166.750000,126.300000,283.540000,317.090000,165.600000,181.690000,35.920000,-33.630000
2,Bengaluru,62.520000,66.310000,72.210000,79.580000,128.440000,116.000000,99.500000,104.120000,18.930000,-27.280000
3,Bhopal,117.130000,90.230000,91.150000,115.760000,216.550000,202.920000,207.690000,165.830000,23.420000,1.170000
4,Chennai,70.400000,70.000000,67.860000,71.280000,159.670000,112.630000,111.250000,127.640000,20.060000,-1.250000
5,Delhi,252.670000,156.160000,166.980000,236.610000,313.410000,288.140000,324.550000,282.860000,9.750000,6.350000
6,Ghaziabad,204.400000,154.230000,127.190000,93.250000,287.590000,271.900000,266.920000,215.050000,25.220000,54.380000
7,Gurugram,136.680000,97.400000,104.670000,134.670000,267.480000,215.560000,216.450000,209.740000,21.590000,1.470000
8,Hyderabad,85.430000,80.800000,64.790000,85.340000,116.910000,108.750000,109.360000,103.600000,11.380000,0.100000
9,Indore,108.940000,97.970000,84.960000,85.230000,159.000000,181.750000,127.950000,116.500000,26.730000,21.770000
